In [1]:
from warnings import simplefilter
# ignore all future warnings / convergence warnings, !!! In a real workflow do not do this !!!
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)
import descarteslabs as dl
import numpy as np
import pickle
from descarteslabs.client.services import Places
import pandas as pd
import matplotlib as mpl
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import descarteslabs.workflows as wf
import shapely.geometry
import shapely.ops
import shapely.prepared
import rasterio.features
from tqdm.notebook import tqdm
import msgpack
import msgpack_numpy
%matplotlib inline

In [2]:
import datetime
def get_yday(tile_info):
    date_t = tile_info['group']
    return datetime.datetime.strptime(f"{date_t[0]}-{date_t[1]}-{date_t[2]}",
                               "%Y-%m-%d").timetuple().tm_yday
def get_yday_arrays(data, info):
    # Assume single year of data
    days = np.array([get_yday(t) - 1 for t in info])
    yday_array = np.zeros((365, data.shape[1], data.shape[2], data.shape[3]))
    yday_array[days] = data
    return yday_array

def get_monthly_arrays(data, info):
    years = sorted(list(set([x['group'][0] for x in info])))
    months = sorted(list(set([x['group'][1] for x in info])))
    dates_ = list(zip(np.arange(data.shape[0], dtype=int), [x['group'] for x in info]))
    date_ranges = {(x,y): [] for x in years for y in range(1,13)}
    for d in dates_:
        date_ranges[d[1][:2]].append(d[0])
    avg_array = np.zeros((12*len(years), data.shape[1], data.shape[2], data.shape[3]))
    for k, dr in enumerate(sorted([(x,y) for x in years for y in months])):
        if date_ranges[dr]:
            avg_array[k] = data[date_ranges[dr][0]:date_ranges[dr][-1]+1].mean(axis=0)
    return avg_array

def msgpack_to_numpy(byte_array):
    return msgpack.unpackb(byte_array, object_hook=msgpack_numpy.decode)[0]

def process_cdl(cdl_r, valid_crops):
    flat = np.array(cdl_r).reshape(512*512)
    flat = pd.Series(flat).apply(lambda v: v * (int(v) in valid_crops))
    flat = flat.apply(lambda v: crops_enc[int(v)])
    return flat.values.reshape((512,512))

def get_dnn_data(img, crop, valid_crops):
    crop = process_cdl(crop, valid_crops)
    nz = np.where(crop > 0)
    res_y = crop[nz]
    res_x = img[:,:,nz[0],nz[1]].transpose((2,0,1))
    return res_x, res_y

In [3]:
'''sac = shapely.geometry.shape(
    dl.places.shape("north-america_united-states_california_sacramento-valley").geometry
)
sj = shapely.geometry.shape(
    dl.places.shape(
        "north-america_united-states_california_san-joaquin-valley"
    ).geometry
)
central_valley_aoi = sac.union(sj)
'''
central_valley_aoi = dl.places.shape("north-america_united-states_california_san-joaquin-valley_stanislaus")
tiles = dl.scenes.DLTile.from_shape(
    central_valley_aoi, resolution=15, tilesize=512, pad=0
)
print(len(tiles))
start_datetime = "2019-01-01"
end_datetime = "2020-01-01"

98


In [4]:
def cloud_masked_daily_product(product_id: str, start_datetime: str, end_datetime: str) -> wf.ImageCollection:
    "Get a product by ID, masked by the DL cloud mask and mosaicked by day"
    if 'airbus' in product_id.lower():
        cloud = 'derived:visual_cloud_mask'
    elif 'sentinel' in product_id.lower():
        cloud = "cloud-mask"
    else:
        cloud = "valid-cloudfree"
    ic = wf.ImageCollection.from_id(product_id, start_datetime, end_datetime)
    cloudmask = (
        wf.ImageCollection.from_id(
            product_id, start_datetime, end_datetime
        ).pick_bands(cloud)
        == 0
    )

    # Make an ImageCollectionGroupby object, for quicker lookups from `ic` by date (you can use it like a dict)
    ic_date_groupby = ic.groupby(dates=("year", "month", "day"))
    # For each cloudmask date, pick the corresponding image from `ic` by date, mosiac both, and mask them.
    # (Not all scenes have cloudmasks processed, so this ensures we only return scenes that do.)
    return cloudmask.groupby(dates=("year", "month", "day")).map(
        lambda ymd, mask_imgs: ic_date_groupby[ymd].mosaic().mask(mask_imgs.mosaic())
    )

def ndvi(ic: wf.ImageCollection) -> wf.ImageCollection:
    nir, red = ic.unpack_bands("nir red")
    ndvi = (nir - red) / (nir + red)
    return ndvi.rename_bands("ndvi")

def isin(ic: wf.ImageCollection, values: list) -> wf.ImageCollection:
    "Like np.isin, for Workflows"
    assert len(values) > 0
    result = False
    for value in values:
        result = result | (ic == value)
    return result

```X_fp = r'/mnt/u_net_3d/saved_models/saved_data/x_rf.npy'
y_fp = r'/mnt/u_net_3d/saved_models/saved_data/y_rf.npy'
X_t = np.load(X_fp)
y_t = np.load(y_fp)```

In [5]:
grains_oils_grass_beans = [1,2,3,4,5,6,10,11,12,13,21,22,23,24,25,26,27,28,29,
                        30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,51,
                        52,53,225,226,228,230,232,234,235,236,237,238,239,240,241,254]

deli_crops = [14, 48, 49, 50, 54, 55, 57, 206, 207, 208, 209, 213, 214, 216,
            219, 221, 222, 224, 227, 229, 231, 242, 243, 244, 245, 246, 247,
            248, 249, 250]

tree_crops = [66, 67, 68, 69, 72, 74, 75, 76, 77, 204, 210, 211, 212, 215, 217,
            218,220, 223]

crops_list = deli_crops + tree_crops
crops_enc = {id_: k + 1 for k, id_ in enumerate(sorted(crops_list))}
crops_enc[0] = 0
crops_dec = {crops_enc[id_]: id_ for id_ in crops_enc}
def process_crop_split(x):
    split = x.split()
    crop = ''
    for s in split:
        if not s.isnumeric():
            crop += f"{s} "
        else:
            return crop[:-1], int(s)
# replace ... with full path on your machine
fp = r'/mnt/u_net_3d/saved_models/saved_data/label_map.txt'
text = ""
with open(fp, 'r') as file:
    for line in file.read():
        text += line
text = text.split('\n')[3:]
crop_label_enc = {process_crop_split(x)[1]: process_crop_split(x)[0] for x in text}
crop_label_dec = {crop_label_enc[k]: k for k in crop_label_enc}
#y_t = np.array([crop_label_enc[crops_dec[y_]] for y_ in y_t])
#X_l = X_t.reshape(X_t.shape[0], 12, 5)

In [6]:
crop_one_hot = {crop: k for k, crop in enumerate(sorted(list(crop_label_dec)))}
crop_one_hot_dec = {crop_one_hot[c]: c for c in crop_one_hot}
'''y_l = np.zeros((y_t.shape[0], 78))
for k, y in enumerate(y_t):
    y_l[k][crop_one_hot[y]] = 1'''

'y_l = np.zeros((y_t.shape[0], 78))\nfor k, y in enumerate(y_t):\n    y_l[k][crop_one_hot[y]] = 1'

In [7]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_l, y_l, test_size=0.2, random_state=2020)

In [10]:
start_datetime = "2019-01-01"
end_datetime = "2020-01-01"

l8_daily = cloud_masked_daily_product(
    "landsat:LC08:01:T1:TOAR", start_datetime, end_datetime
).pick_bands("red green blue nir swir1")

s2_daily = cloud_masked_daily_product(
    "sentinel-2:L1C", start_datetime, end_datetime
).pick_bands("red green blue nir swir1")

l8_with_ndvi = l8_daily.concat_bands(ndvi(l8_daily))
ab_ic = wf.ImageCollection.from_id(
    "airbus:oneatlas:spot:v2", start_datetime=start_datetime, end_datetime=end_datetime
).pick_bands("red green blue")
ab_date_groupby = ab_ic.groupby(dates=("year", "month", "day"))
ab_daily = ab_ic.groupby(dates=("year", "month", "day")).map(
        lambda ymd, mask_imgs: ab_date_groupby[ymd].mosaic())
cdl = wf.ImageCollection.from_id(
    "usda:cdl:v1", start_datetime="2016-01-01", end_datetime="2019-01-01"
).pick_bands("class")
cdl_train = wf.ImageCollection.from_id(
    "usda:cdl:v1", start_datetime=start_datetime, end_datetime=end_datetime
).pick_bands("class")


grains_oils_grass_beans = [1,2,3,4,5,6,10,11,12,13,21,22,23,24,25,26,27,28,29,
                        30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,51,
                        52,53,225,226,228,230,232,234,235,236,237,238,239,240,241,254]

deli_crops = [14, 48, 49, 50, 54, 55, 57, 206, 207, 208, 209, 213, 214, 216,
            219, 221, 222, 224, 227, 229, 231, 242, 243, 244, 245, 246, 247,
            248, 249, 250]

tree_crops = [66, 67, 68, 69, 72, 74, 75, 76, 77, 204, 210, 211, 212, 215, 217,
            218,220, 223]

crops_list = deli_crops + tree_crops
crops_enc = {id_: k + 1 for k, id_ in enumerate(sorted(crops_list))}
crops_enc[0] = 0
crops_dec = {crops_enc[id_]: id_ for id_ in crops_enc}

is_crops = isin(cdl, crops_list)
is_crops_19 = is_crops[-1]

four_year_combo = is_crops.sum(axis="images") + is_crops_19  # double-weight 2019
four_year_binary = four_year_combo >= 2
cdl_mask = ~four_year_binary
l8_masked = l8_with_ndvi.mask(cdl_mask)
central_valley_ctx = dl.scenes.AOI(central_valley_aoi, shape=(2048, 2048), crs="EPSG:4326")
all_cdl = four_year_binary.compute(central_valley_ctx)
all_cdl = msgpack.unpackb(all_cdl, object_hook=msgpack_numpy.decode)
all_cdl['geocontext']["gdal_geotrans"]
shapes = list(
    geom for geom, value in
    rasterio.features.shapes(
        all_cdl['ndarray'].astype("uint8"), 
        transform=rasterio.transform.Affine.from_gdal(*all_cdl['geocontext']["gdal_geotrans"])
    )
    if value == 1
)
print(f"length of shapes: {len(shapes)}")
all_valid = shapely.ops.unary_union([shapely.geometry.shape(s) for s in shapes]).simplify(0.3)
print(f'Type of all_valid: {type(all_valid)}')
all_valid_prepped = shapely.prepared.prep(all_valid)
valid_tiles = [t for t in tqdm(tiles) if all_valid_prepped.intersects(t.geometry)]
print(f'No. Valid Tiles: {len(valid_tiles)}')
print(f'Percentage of valid tiles: {100*(len(valid_tiles) / len(tiles))}')

MethodNotImplemented: 501 

In [ ]:
import keras
print(keras.__version__)
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed
from keras.layers import Dropout, Bidirectional, Concatenate
from keras.layers import BatchNormalization, Conv1D
from keras import backend as K

In [ ]:
input_ = Input((365, 5))
conv = BatchNormalization()(input_)
conv = Conv1D(128, 3, padding='same')(conv)
conv = BatchNormalization()(conv)
conv = Conv1D(128, 3, padding='same')(conv)
conv = Conv1D(128, 3, padding='same')(conv)
conv = BatchNormalization()(conv)
conv = Conv1D(128, 3, padding='same')(conv)
lst = BatchNormalization()(conv)
lst = Bidirectional(LSTM(units=365, recurrent_dropout=0))(lst)
out = Dropout(0.25)(lst)
out = BatchNormalization()(out)
out = Dense(78, activation='softmax')(out)
model = Model(input_, out)
sgd = keras.optimizers.Adam(lr=0.000007, clipvalue=50)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
def get_train_tile(tile, imagery):
    if imagery == 'landsat':
        l8_d = wf.compute([l8_daily.ndarray], #l8_daily.properties, cdl.ndarray], 
                          tile, format="msgpack", progress_bar=False)
        l8_inf = wf.compute([l8_daily.properties], 
                          tile, progress_bar=False)
        cdl_d = wf.compute([cdl_train.ndarray], 
                          tile, format='msgpack', progress_bar=False)
        l8_d = msgpack_to_numpy(l8_d)
        cdl_d = msgpack_to_numpy(cdl_d)
        img_t = get_yday_arrays(l8_d, l8_inf[0])
    elif imagery == 'sentinel2':
        s2_d = wf.compute([s2_daily.ndarray], #l8_daily.properties, cdl.ndarray], 
                          tile, format="msgpack", progress_bar=False)
        s2_inf = wf.compute([s2_daily.properties], 
                          tile, progress_bar=False)
        cdl_d = wf.compute([cdl_train.ndarray], 
                          tile, format='msgpack', progress_bar=False)
        s2_d = msgpack_to_numpy(s2_d)
        cdl_d = msgpack_to_numpy(cdl_d)
        img_t = get_yday_arrays(s2_d, s2_inf[0])
    Xtt, ytt = get_dnn_data(img_t, cdl_d, crops_list)
    ytt = np.array([crop_label_enc[crops_dec[y_]] for y_ in ytt])
    y_res = np.zeros((ytt.shape[0], 78))
    for k, y in enumerate(ytt):
        y_res[k][crop_one_hot[y]] = 1
    return Xtt, y_res

In [ ]:
for k, tile in enumerate(valid_tiles):
    try:
        print(f"Tile {k+1} out of 67")
        X_tt, y_tt = get_train_tile(tile, 'sentinel2')
        X_train, X_test, y_train, y_test = train_test_split(X_tt, y_tt, test_size=0.10, random_state=2020)
        #model.fit(X_train, y_train, batch_size=256, epochs=1, validation_data=[X_test, y_test])
        #K.set_value(model.optimizer.learning_rate, 0.00009)
        model.fit(X_train, y_train, batch_size=min(256, X_tt.shape[0]),
                  epochs=3, validation_data=[X_test, y_test])
        #K.set_value(model.optimizer.learning_rate, 0.0001)
        #X_ttt = np.concatenate((X_train, X_tt))
        #y_ttt = np.concatenate((y_train, y_tt))
        #shuff = np.arange(X_ttt.shape[0])
        #np.random.shuffle(shuff)
        #K.set_value(model.optimizer.learning_rate, 0.00009)
        model.fit(X_tt, y_tt, batch_size=256, epochs=2, validation_data=[X_test, y_test])
    except Exception as e:
        print(e)

In [ ]:
model.save(r'/mnt/u_net_3d/saved_models/model365_1.h5')

In [ ]:
start_datetime = "2018-01-01"
end_datetime = "2019-01-01"
l8_daily = cloud_masked_daily_product(
    "landsat:LC08:01:T1:TOAR", start_datetime, end_datetime
).pick_bands("red green blue nir swir1")
s2_daily = cloud_masked_daily_product(
    "sentinel-2:L1C", start_datetime, end_datetime
).pick_bands("red green blue nir swir1")
y18_true, y18_pred = [], []
for k, tile in enumerate(valid_tiles):
    try:
        X_tt, y_tt = get_train_tile(tile, 'sentinel2')
        cur_pred = model.predict(X_tt).argmax(-1)
        y18_pred += cur_pred.tolist()
        y18_true += y_tt.argmax(-1).tolist()
        print(f"Iteration {k+1} / 67 accuracy: {accuracy_score(y_tt.argmax(-1), cur_pred)}")
    except Exception as e:
        print(e)

In [ ]:
accuracy_score(y18_true, y18_pred)

In [ ]:
#model.fit(X_train, y_train, batch_size=64, epochs=15, validation_data=[X_test, y_test])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
def get_conf_matrix(y_true, y_pred, labels, save=False):
    df = pd.DataFrame(confusion_matrix(y_true, y_pred, labels))
    df.columns = [f'Predicted {lab}' for lab in labels]
    df.index = [f'True {lab}' for lab in labels]
    cmap = sns.light_palette((237, 85, 74), input="husl",as_cmap=True)
    plt.figure(figsize=(30,30))
    sns.heatmap(df,annot=True, fmt='d', cbar=0, cmap=cmap, annot_kws={"size": 20})
    if save:
        plt.savefig(save)
    plt.show()

In [ ]:
preds = [crop_one_hot_dec[y] for y in model.predict(X_test).argmax(-1)]
y_true = [crop_one_hot_dec[y] for y in y_test.argmax(-1)]

In [ ]:
print(accuracy_score(y_true, preds))
get_conf_matrix(y_true, preds, [crop for crop in list(crop_one_hot) if crop in y_true])

In [ ]:
start_datetime = "2018-01-01"
end_datetime = "2019-01-01"
l8_daily = cloud_masked_daily_product(
    "landsat:LC08:01:T1:TOAR", start_datetime, end_datetime
).pick_bands("red green blue nir swir1")
l8_with_ndvi = l8_daily.concat_bands(ndvi(l8_daily))
ab_ic = wf.ImageCollection.from_id(
    "airbus:oneatlas:spot:v2", start_datetime=start_datetime, end_datetime=end_datetime
).pick_bands("red green blue")
ab_date_groupby = ab_ic.groupby(dates=("year", "month", "day"))
ab_daily = ab_ic.groupby(dates=("year", "month", "day")).map(
        lambda ymd, mask_imgs: ab_date_groupby[ymd].mosaic())
cdl = wf.ImageCollection.from_id(
    "usda:cdl:v1", start_datetime="2018-01-01", end_datetime="2019-01-01"
).pick_bands("class")

grains_oils_grass_beans = [1,2,3,4,5,6,10,11,12,13,21,22,23,24,25,26,27,28,29,
                        30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,51,
                        52,53,225,226,228,230,232,234,235,236,237,238,239,240,241,254]

deli_crops = [14, 48, 49, 50, 54, 55, 57, 206, 207, 208, 209, 213, 214, 216,
            219, 221, 222, 224, 227, 229, 231, 242, 243, 244, 245, 246, 247,
            248, 249, 250]

tree_crops = [66, 67, 68, 69, 72, 74, 75, 76, 77, 204, 210, 211, 212, 215, 217,
            218,220, 223]

crops_list = deli_crops + tree_crops
crops_enc = {id_: k + 1 for k, id_ in enumerate(sorted(crops_list))}
crops_enc[0] = 0
crops_dec = {crops_enc[id_]: id_ for id_ in crops_enc}

is_crops = isin(cdl, crops_list)
is_crops_19 = is_crops[-1]

four_year_combo = is_crops.sum(axis="images") + is_crops_19  # double-weight 2019
four_year_binary = four_year_combo >= 2
cdl_mask = ~four_year_binary

l8_masked = l8_with_ndvi.mask(cdl_mask)
central_valley_ctx = dl.scenes.AOI(central_valley_aoi, shape=(2048, 2048), crs="EPSG:4326")
all_cdl = four_year_binary.compute(central_valley_ctx, format='msgpack')

In [ ]:
test[0][0, :3].shape

In [ ]:
plt.imshow(np.transpose(test[0].mean(0)[:3], (1,2,0)))

In [ ]:
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral, create_pairwise_gaussian
import cv2
from skimage.color import gray2rgb
from skimage.color import rgb2gray
#Original_image = Image which has to labelled
#Annotated image = Which has been labelled by some technique( FCN in this case)
#Output_image = Name of the final output image after applying CRF
#Use_2d = boolean variable 
#if use_2d = True specialised 2D fucntions will be applied
#else Generic functions will be applied

def crf(original_image, annotated_image,output_image, use_2d = True):
    
    # Converting annotated image to RGB if it is Gray scale
    if(len(annotated_image.shape)<3):
        annotated_image = gray2rgb(annotated_image).astype(np.uint32)
    
    #cv2.imwrite("testing2.png",annotated_image)
    annotated_image = annotated_image.astype(np.uint32)
    #Converting the annotations RGB color to single 32 bit integer
    annotated_label = annotated_image[:,:,0].astype(np.uint32) + (annotated_image[:,:,1]<<8).astype(np.uint32) + (annotated_image[:,:,2]<<16).astype(np.uint32)
    
    # Convert the 32bit integer color to 0,1, 2, ... labels.
    colors, labels = np.unique(annotated_label, return_inverse=True)
    
    #Creating a mapping back to 32 bit colors
    colorize = np.empty((len(colors), 3), np.uint8)
    colorize[:,0] = (colors & 0x0000FF)
    colorize[:,1] = (colors & 0x00FF00) >> 8
    colorize[:,2] = (colors & 0xFF0000) >> 16
    
    #Gives no of class labels in the annotated image
    n_labels = len(set(labels.flat)) 
    
    print("No of labels in the Image are ")
    print(n_labels)
    
    
    #Setting up the CRF model
    if use_2d :
        d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

        # get unary potentials (neg log probability)
        U = unary_from_labels(labels, n_labels, gt_prob=0.55, zero_unsure=False)
        d.setUnaryEnergy(U)

        # This adds the color-independent term, features are the locations only.
        d.addPairwiseGaussian(sxy=(5, 5), compat=5,
                              kernel=dcrf.FULL_KERNEL,
                          normalization=dcrf.NORMALIZE_SYMMETRIC)

        # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
        d.addPairwiseBilateral(sxy=(40, 40), srgb=(5, 5, 5), rgbim=original_image,
                           compat=25,
                           kernel=dcrf.FULL_KERNEL,
                           normalization=dcrf.NORMALIZE_SYMMETRIC)
        
    #Run Inference for 5 steps 
    Q = d.inference(5)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    # Convert the MAP (labels) back to the corresponding colors and save the image.
    # Note that there is no "unknown" here anymore, no matter what we had at first.
    MAP = colorize[MAP,:]
    cv2.imwrite(output_image,MAP.reshape(original_image.shape))
    return MAP.reshape(original_image.shape)